<a href="https://colab.research.google.com/github/rigonzal/ANN/blob/master/Approach2/Pregunta_3_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INF-395 Redes Neuronales Artificiales II-2018

## Tarea 2 - Redes Convolucionales y sus aplicaciones


26/11/2018

* Rodrigo González Smith 201303026-2
* Ignacio Valenzuela Albornoz 2014073055-1

## Introducción Parte 3



Débido al tema del trabajo usaremos **Keras** para la creación de las redes y su posterior entrenamiento/validación. Como también las librerías comunes de trabajo de Python que nos facilitarán nuestras tareas experimentales:

In [0]:
#Correr solo en Colab
!pip install -q keras
!pip install -q nltk
!pip install -U -q PyDrive

In [2]:
import numpy as np   
import os, re, sys
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, rmsprop
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
import h5py
import nltk
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
nltk.download('stopwords')
nltk.download('wordnet')

%matplotlib inline

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.




---


## 1) CNN sobre texto

Ahora usaremos redes neuronales convolucionales para (NLP), es decir, procesamiento de Lenguaje Natural. El dataset cuenta con miles de avisos de trabajo, y el objetivo es a partir de estos poder predecir el salario asociado. Como bien dice el [sitio oficial](https://www.kaggle.com/c/job-salary-prediction)  que entrega el dataset, es crear un *Salary prediction engine*. 

Primero que todos haremos una conexión con el google drive para poder importar los archivos directamente desde allí y acelerar el proceso de carga de datos.

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
file_list = drive.ListFile().GetList()  # Analizamos el contenido del Drive para obtener el id del archivo
for file in file_list:
  if(file['title'] == 'Train_rev1.csv' or 'Val' in file['title'] ):
    print('title {}, id {}'.format(file['title'], file['id']))

title Train_rev1.csv, id 1K--wMccLW0eHdVrp8RXe3c-bXPfLNPxo
title Train_rev1.csv, id 1bIyTdMaZn5h0EP0eNFmkFpblI0mBTT-O
title Valid_rev1.csv, id 1NFYbS6ulPjdjQJRK5ixEE7rT9bQ7zNKN
title CV-Ignacio-Valenzuela.pdf, id 1cicPdaP4SpPmc9UQ8BPFQ_Shg9YFXiqz
title Ignacio Valenzuela, id 0B_SDEfD2s3nUVDF2YThQNFV5S3c


In [0]:
download_path = os.path.expanduser('~/data') # Luego creamos un directorio local para almacenar la data
try:
  os.makedirs(download_path)
except FileExistsError:
  pass

In [0]:
output_file = os.path.join(download_path, 'Train_rev1.csv') # Luego importamos la informacion

temp_file = drive.CreateFile({'id': '1K--wMccLW0eHdVrp8RXe3c-bXPfLNPxo'})
temp_file.GetContentFile(output_file)

In [0]:
data = pd.read_csv(output_file)

In [9]:
data.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


Con el código anterior, se logró cargar el dataset a pandas en un tiempo muy corto. Este procedimiento permite una carga de datos directamente desde GoogleDrive, y como Colab trabaja dentro de ese entorno, el procedimiento es simple y rápido.

Ahora se implementará una función la cual preprocesará la información. Primero se dejarán todas las palabras en minúscula, luego gracias a funciones regulares quitaremos ciertos patrones de los anuncios (que estan dentro de la columna "FullDescription") el analiza la presencia de números y otros símbolos reemplazandolo por un espacio en blanco. Luego se aplica a la eliminación de stopwords aplicación de lemmatization. Luego se guarda el dato procesado y se actualiza la columna dentro del dataset ya que esta será la que ocuparemos para realizar el split de los datos.

In [0]:
stoplist = set(stopwords.words('english'))

def pre_procesar(df):
    #preprocesar texto de los anuncios
    #Eliminación de stopwords, stemming/lemmatization, puntuación, etc
    df_procesado = {}
    for column in df.columns:
      columna = list()
      for s in df[column]:
        s= s.lower()
        s= re.sub(r'[^\w]', ' ',s)
        s= re.sub(r'\b[a-z]\b', ' ',  s)
        s= re.sub(r'\b[a-z][a-z]\b', ' ',  s)
        s=re.sub(r'\b[0-9]\b', ' ',  s)
        s=re.sub(r'\b[0-9][0-9]\b', ' ',  s)
        s=re.sub(r'\b[0-9][0-9][0-9]\b', ' ',  s)
        s= re.sub(r'[^\w.]', ' ', s)
        s= list(s.split())
        s= [word for word in s if not word in stoplist]
        s= [nltk.WordNetLemmatizer().lemmatize(word) for word in s]
        columna.append(' '.join(s))
      df_procesado[column] = columna
    return pd.DataFrame(df_procesado)

Después de hacer el resumen del dataset, vemos que posee variables con valor NaN. Estos no son strings, si no que valores int, por lo que aplicar ciertas funciones de preprocesamiento no podrán aplicarse por formato erróneo. Para eso, todas las variables que posean NaN las pasaremos a un string "nan", para que así no hayan problemas de formato.

In [0]:
y_dataframe=data['SalaryNormalized'].values
x_dataframe=data.drop(columns=['SalaryNormalized', 'SalaryRaw', 'Id'])

x_dataframe.loc[x_dataframe['ContractType'].isna(), 'ContractType'] = 'nan'
x_dataframe.loc[x_dataframe['ContractTime'].isna(), 'ContractTime'] = 'nan'
x_dataframe.loc[x_dataframe['Company'].isna(), 'Company'] = 'nan'
x_dataframe.loc[x_dataframe['Title'].isna(), 'Title'] = ''
x_dataframe.loc[x_dataframe['SourceName'].isna(), 'SourceName'] = 'nan'

In [12]:
x_dataframe.head()

,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SourceName
0,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,nan,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
1,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,nan,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
2,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,nan,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
3,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,nan,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
4,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,nan,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk


Luego de esto, usamos la función que preprocesa el texto y posteriormente realizamos un split de la informacion para crear set de entrenamiento, validación y testeo.

In [0]:
df_procesado = pre_procesar(x_dataframe)
k=len(df_procesado)
y_train = y_dataframe[0:int(k*0.70)]
y_val = y_dataframe[int(k*0.70):int(k*0.85)]
x_train=df_procesado[0:int(k*0.70)] #70% training
x_val=df_procesado[int(k*0.70):int(k*0.85)] #15% validation
x_test=df_procesado[int(k*0.85):] #15% test

In [14]:
df_procesado.head()

,Category,Company,ContractTime,ContractType,FullDescription,LocationNormalized,LocationRaw,SourceName,Title
0,engineering job,gregory martin international,permanent,nan,engineering system analyst dorking surrey sala...,dorking,dorking surrey surrey,library,engineering system analyst
1,engineering job,gregory martin international,permanent,nan,stress engineer glasgow salary currently looki...,glasgow,glasgow scotland scotland,library,stress engineer glasgow
2,engineering job,gregory martin international,permanent,nan,mathematical modeller simulation analyst opera...,hampshire,hampshire south east south east,library,modelling simulation analyst
3,engineering job,gregory martin international,permanent,nan,engineering system analyst mathematical modell...,surrey,surrey south east south east,library,engineering system analyst mathematical modeller
4,engineering job,gregory martin international,permanent,nan,pioneer miser engineering system analyst dorki...,surrey,surrey south east south east,library,pioneer miser engineering system analyst


Vemos que el dataset procesado almacenó correctamente las palabras, separadas mediante un espacio, así tomamos las palabras mas significativas de cada anuncio y permitirá un mejor análisis del texto. 

Ahora procederemos a usar embedding con los textos para mapearlos en vectores y así poder implementarlos en un modelo de red convolucional. Eso sí, en vez de crear los vectores propios, implementaremos los del archivo Glove ya que estos fueron creados mediante un entrenamiento de muchos datos, por lo que será un buen modelo a implementar. Cabe destacar que la dimensionalidad que aplica es de 100.

In [0]:
word_index=dict()
j=0
for frase in x_train:
    seq=frase.split()
    for term in seq:
        if term not in word_index.keys():
            word_index[term]=j
            j+=1

In [17]:
for file in file_list:
  if('glove' in file['title']):
    print('title {}, id {}'.format(file['title'], file['id']))

title glove.6B.100d.txt, id 11oaqviHAKGl281V1PqTxCz-dMThf553W


In [0]:
f = os.path.join(download_path, 'glove.6B.100d.txt') # Luego importamos la informacion

temp_file = drive.CreateFile({'id': '11oaqviHAKGl281V1PqTxCz-dMThf553W'})
temp_file.GetContentFile(f)

Lo anterior es el mismo proceso anterior, donde se busca el archivo, se descarga y se implementa en una variable. Luego, se abre este archivo de texto y se aplica el embedding para formar el índice de vectores.

In [19]:
embeddings_index = {}
f = open(f)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Se encontraron %s terminos con sus vectores de embedding.' % len(embeddings_index))

Se encontraron 400000 terminos con sus vectores de embedding.


Luego de eso, se procede a armar la matriz de los vectores.

In [0]:
embedding_vector=100
embedding_matrix = np.zeros((len(word_index.keys()), embedding_vector))   #puede probar otra inicialización

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words sin match en Glove, serán vectores de ceros.
        embedding_matrix[i] = embedding_vector

Ahora llego el momento de crear un modelo y aplicarlo con estos datos. 

In [26]:
"""Definir input para el modelo: """
from keras.preprocessing import sequence
from keras.layers import Input
from keras.models import Model

x_new_train = [[word_index[word] for word in text.split()] for text in x_train]
x_new_val = [[word_index[word] for word in text.split() if word in word_index] for text in x_val]

max_input_lenght = 10 #modificar este valor en base a su experimentación
Xtrain = sequence.pad_sequences(x_new_train,maxlen=max_input_lenght,padding='post',value=0)
Xval = sequence.pad_sequences(x_new_val,maxlen=max_input_lenght,padding='post',value=0)

"""Define model trough Model API in Keras"""
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPooling1D
embedding_vector=100
embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                     input_length=max_input_lenght,trainable=False)

sequence_input = Input(shape=(max_input_lenght,))
embedded_sequences = embedding_layer(sequence_input)
cov1= Conv1D(128, 5, activation='relu',padding='same')(embedded_sequences)
pool = MaxPooling1D(pool_size=2)(cov1)
flat = Flatten()(pool)
preds = Dense(1, activation='linear')(flat)
model = Model(sequence_input, preds)
model.summary()

model.compile(loss='mse',optimizer='rmsprop',metrics=['acc'])
model.fit(Xtrain, y_dataframe, validation_data=(Xval, y_val),epochs=25, batch_size=256)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 10, 100)           900       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 10, 128)           64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 5, 128)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 641       
Total params: 65,669
Trainable params: 64,769
Non-trainable params: 900
_________________________________________________________________


ValueError: ignored